In [1]:
# Import necessary Libraries.
from __future__ import division # will change the / operator to mean true division throughout the module.
import os 
import pandas as pd # Pandas Library (will use to load data,create data frame,perform operation on data frame...etc).
import numpy as np # Numpy Library ( will use to convert data frame to array or creating array etc...).
import pyodbc # It is used to make a connection to SQL Server and fecth the data.
import re # For regular expressions.
import gc # For cleaning memory(Removes unreferenced objects).
from itertools import permutations # For permutation.
from itertools import combinations # For combinations.
import pickle # It will use to convert dataframe to pickle file and load/read pickle file.
from pandas import ExcelWriter # Dumping the data into excel sheet.
import seaborn as sb # For visualization (bar,box,hist,reg plot...etc).
import itertools # For iterating lists, tuples, dicts, and sets...etc.
from datetime import datetime # For date and time value.
import win32com.client # To send an email.
from tkinter import * # For dialog box.
import tkinter.simpledialog
import tkinter.messagebox
import openpyxl
import warnings # Igonre warnings
warnings.filterwarnings('ignore')

# Get BrewData data lake connection using MFA authentication type.
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};" # Driver info,select driver version according to available driver in your machine.
                      #"Driver={SQL Server Native Client 11.0};"
                      "Server=edhazurencertdmgbprod.database.windows.net;" # Server/Host name.
                      "Database=edhazurencertdmgbprod;" # Database name.
                      "Trusted_Connection=NO;" # If you want to use windows authetication then keep flag as Yes else No.
                      "Authentication=ActiveDirectoryInteractive;" # Authentication type(MFA).
                      "UID=tejas.c@ab-inbev.com;" # User id for connecting to database.
                      #"PWD=;" # Password for connecting to databse.
                     )
cursor = cnxn.cursor()
start_date = '2022-01-01'
end_date = '2022-06-13'
query_EKKO = '''
Select EBELN as 'Pur. Doc.', LIFNR as 'Vendor'
from abi_compliance_control_us_non_sensitive_rpt.EKKO
where LIFNR in (8341583)
'''
df_EKKO = pd.read_sql_query(query_EKKO,cnxn)
df_EKKO = df_EKKO[["Pur. Doc."]]
x=df_EKKO["Pur. Doc."].unique().tolist()
y=tuple(x)
query_EKPO = '''
Select EBELN as 'Pur. Doc.', EBELP as 'Item', LOEKZ as 'D', MENGE as 'PO Quantity', BRTWR as 'Gross value', AEDAT as 'Creation Date'
from abi_compliance_control_us_non_sensitive_rpt.EKPO 
where AEDAT>='''+'\''+start_date+'\''+'''
and AEDAT<='''+'\''+end_date+'\''+''' 
and EBELN in {}'''.format(y)   
df_EKPO = pd.read_sql_query(query_EKPO,cnxn)
df_EKPO=df_EKPO.drop_duplicates()
a=df_EKPO["Pur. Doc."].unique().tolist()
b=tuple(a)
query_EKBE = '''
Select P1.EBELN as 'Pur. Doc.',P1.EBELP as 'Item',P1.BUDAT as 'Pstng Date',
P1.WRBTR as 'Amount',P1.MENGE as Quantity, P1.BEWTP as 'HCt', P1.SHKZG as 'D/C'
from abi_compliance_control_us_non_sensitive_rpt.EKBE as P1
where P1.EBELN in {}'''.format(b)
df_EKBE = pd.read_sql_query(query_EKBE,cnxn)

df_AmazonShipments=pd.read_excel("C:\\Users\\40107500\\OneDrive - Anheuser-Busch InBev\\Attachments\\Amazon_shipments.xls")
df_AmazonOrder=pd.read_excel("C:\\Users\\40107500\\OneDrive - Anheuser-Busch InBev\\Attachments\\Amazon_full_orders.xls")
df_AmazonRefunds = pd.read_excel("C:\\Users\\40107500\\OneDrive - Anheuser-Busch InBev\\Attachments\\Amazon_Refunds.xls")

parked_invoices=df_EKBE[(df_EKBE["HCt"]=='N') | (df_EKBE["HCt"]=='T')]
parked_invoices=parked_invoices[["HCt","D/C", "Pur. Doc.", "Item"]]
parked_invoices.rename(columns={'HCt':'Parked Invoice'}, inplace=True)
parked_invoices.rename(columns={'D/C':'PI Type'}, inplace=True)
parked_invoices["Pur. Doc."] = parked_invoices["Pur. Doc."].astype(str)
parked_invoices["Item"]=parked_invoices["Item"].astype(str)
parked_invoices.rename(columns={'Pur. Doc.':'PO Number'}, inplace=True)
parked_invoices.rename(columns={'Item':'PO Line Item Id'}, inplace=True)
parked_invoices["PO Line Item Id"] =parked_invoices["PO Line Item Id"] + ".0"

df_AmazonOrder["PO Line Item Id"]=df_AmazonOrder["PO Line Item Id"].astype(str)
df_AmazonOrder['PO Number'] = df_AmazonOrder['PO Number'].str[3:]
df_AmazonShipments["PO Line Item Id"]=df_AmazonShipments["PO Line Item Id"].astype(str)
df_AmazonShipments['PO Number'] = df_AmazonShipments['PO Number'].str[3:]
df_AmazonRefunds["PO Line Item Id"]=df_AmazonRefunds["PO Line Item Id"].astype(str)
df_AmazonRefunds['PO Number'] = df_AmazonRefunds['PO Number'].str[3:]

df_AmazonOrder = df_AmazonOrder[["PO Number" ,"PO Line Item Id", "Invoice Status", "Order Status", "Order Date", "Order Net Total", "Order Quantity"]]
df_AmazonShipments=df_AmazonShipments[["Account Group", "PO Number", "PO Line Item Id", 
                                        "Shipment Date", "Shipment Status", "Delivery Status"
                                      ,"Expected Delivery Date"]]
df_AmazonRefunds=df_AmazonRefunds[["PO Number","Refund Date", "Refund Status","Refund Total", "PO Line Item Id"]]

df_EKPO["Pur. Doc."] = df_EKPO["Pur. Doc."].astype(str)
df_EKPO["Item"]=df_EKPO["Item"].astype(str)
df_EKPO.rename(columns={'Pur. Doc.':'PO Number'}, inplace=True)
df_EKPO.rename(columns={'Item':'PO Line Item Id'}, inplace=True)
df_EKPO["PO Plus Items"]=[f"{a}{b}" for a,b in zip(df_EKPO["PO Number"],df_EKPO["PO Line Item Id"])]
df_EKPO["PO Line Item Id"] =df_EKPO["PO Line Item Id"] + ".0"

df_EKBE["Quantity"]= np.where((df_EKBE["D/C"]=="H"), df_EKBE["Quantity"] * (-1), df_EKBE["Quantity"])
df_EKBE["Amount"]= np.where((df_EKBE["D/C"]=="H"), df_EKBE["Amount"] * (-1), df_EKBE["Amount"])
df_EKBE["Pur. Doc."] = df_EKBE["Pur. Doc."].astype(str)
df_EKBE["Item"]=df_EKBE["Item"].astype(str)
GR=df_EKBE[df_EKBE["HCt"]=='E'].groupby(['Pur. Doc.', 'Item']).sum()
IR=df_EKBE[df_EKBE["HCt"]=='Q'].groupby(['Pur. Doc.', 'Item']).sum()
GR.reset_index(inplace=True)
IR.reset_index(inplace=True)

a=df_EKPO.merge(df_AmazonOrder, on=["PO Number", "PO Line Item Id"], how='left')
a=a.merge(df_AmazonShipments, on=["PO Number", "PO Line Item Id"], how = "left")
a=a.drop_duplicates()
GR["Pur. Doc."] = GR["Pur. Doc."].astype(str)
GR["Item"] = GR["Item"].astype(str)
GR.rename(columns={'Pur. Doc.':'PO Number'}, inplace=True)
GR.rename(columns={'Item':'PO Line Item Id'}, inplace=True)
GR.rename(columns={'Quantity':'GR Quantity'}, inplace=True)
GR.rename(columns={'Amount':'GR Amount'}, inplace=True)
IR["Pur. Doc."] = IR["Pur. Doc."].astype(str)
IR["Item"] = IR["Item"].astype(str)
IR.rename(columns={'Pur. Doc.':'PO Number'}, inplace=True)
IR.rename(columns={'Item':'PO Line Item Id'}, inplace=True)
IR.rename(columns={'Quantity':'IR Quantity'}, inplace=True)
IR.rename(columns={'Amount':'IR Amount'}, inplace=True)
GR["PO Line Item Id"] =GR["PO Line Item Id"] + ".0"
IR["PO Line Item Id"] =IR['PO Line Item Id'] +".0"
b=a.merge(GR,on=["PO Number", "PO Line Item Id"], how='left')
c=b.merge(IR, on=["PO Number", "PO Line Item Id"], how='left')
c=c.merge(parked_invoices, on=["PO Number", "PO Line Item Id"] , how='left')
c=c.merge(df_AmazonRefunds, on=["PO Number", "PO Line Item Id"], how='left')

df_AmazonReturn=pd.read_excel("C:\\Users\\40107500\\OneDrive - Anheuser-Busch InBev\\Attachments\\Amazon_returns_new.xls")
df_AmazonReturn['PO Number'] = df_AmazonReturn['PO Number'].str[3:]
df_AmazonReturn["PO Line Item Id"]=df_AmazonReturn["PO Line Item Id"].astype(str)
df_AmazonReturn.drop(["Order Date"], axis=1, inplace=True)
d=c.merge(df_AmazonReturn, on=["PO Number", "PO Line Item Id"], how='left')
ES=df_EKBE[(df_EKBE["D/C"]=='S') & (df_EKBE["HCt"]=='E')]
ES.drop(["Quantity", "Amount", "Pstng Date"], axis=1, inplace=True)
ES.rename(columns={"Pur. Doc.":"PO Number"}, inplace=True)
ES.rename(columns={"Item":"PO Line Item Id"}, inplace=True)
ES["PO Line Item Id"]=ES["PO Line Item Id"].astype(str)
ES["PO Number"]=ES["PO Number"].astype(str)
ES["PO Line Item Id"] =ES["PO Line Item Id"] + ".0"
EH=df_EKBE[(df_EKBE["D/C"]=='H') & (df_EKBE["HCt"]=='E')]
EH.rename(columns={"Pur. Doc.":"PO Number"}, inplace=True)
EH.rename(columns={"Item":"PO Line Item Id"}, inplace=True)
EH.rename(columns={"Amount":"GR Return Amount"}, inplace=True)
EH.rename(columns={"Quantity":"GR Return Quantity"}, inplace=True)
EH.rename(columns={"Pstng Date":"GR return Posting Date"}, inplace=True)
EH["PO Line Item Id"]=EH["PO Line Item Id"].astype(str)
EH["PO Number"]=EH["PO Number"].astype(str)
EH["PO Line Item Id"] =EH["PO Line Item Id"] + ".0"
g=ES.merge(EH, on=["PO Number", "PO Line Item Id"], how='left' )
g.drop(["HCt_y"], axis=1, inplace=True)
g.rename(columns={'HCt_x':'GR'} , inplace=True)
g.rename(columns={'D/C_y':'GR Return'} , inplace=True)
g.rename(columns={'D/C_x':'GR posted'} , inplace=True)
QS=df_EKBE[(df_EKBE["D/C"]=='S') & (df_EKBE["HCt"]=='Q')]
QS.drop(["Quantity", "Amount", "Pstng Date"], axis=1, inplace=True)
QS.rename(columns={"Pur. Doc.":"PO Number"}, inplace=True)
QS.rename(columns={"Item":"PO Line Item Id"}, inplace=True)
QS["PO Line Item Id"]=QS["PO Line Item Id"].astype(str)
QS["PO Number"]=QS["PO Number"].astype(str)
QS["PO Line Item Id"] =QS["PO Line Item Id"] + ".0"
QH=df_EKBE[(df_EKBE["D/C"]=='H') & (df_EKBE["HCt"]=='Q')]
QH.rename(columns={"Pur. Doc.":"PO Number"}, inplace=True)
QH.rename(columns={"Item":"PO Line Item Id"}, inplace=True)
QH.rename(columns={"Amount":"IR Return Amount"}, inplace=True)
QH.rename(columns={"Quantity":"IR Return Quantity"}, inplace=True)
QH.rename(columns={"Pstng Date":"IR Return Posting Date"}, inplace=True)
QH["PO Line Item Id"]=QH["PO Line Item Id"].astype(str)
QH["PO Number"]=QH["PO Number"].astype(str)
QH["PO Line Item Id"] =QH["PO Line Item Id"] + ".0"
h=QS.merge(QH, on=["PO Number", "PO Line Item Id"], how='left' )
h.drop(["HCt_y"], axis=1, inplace=True)
h.rename(columns={'HCt_x':'IR'} , inplace=True)
h.rename(columns={'D/C_y':'IR Return'} , inplace=True)
h.rename(columns={'D/C_x':'IR posted'} , inplace=True)
i=d.merge(g, on=["PO Number", "PO Line Item Id"], how='left')
j=i.merge(h, on=["PO Number", "PO Line Item Id"], how='left')
k=j.drop_duplicates()
k.to_csv("AbsoluteFinal.csv") 
